In [1]:
%load_ext autoreload
%autoreload 2
from datasets import DatasetDict, load_dataset


lince_pos = load_dataset("lince", "pos_spaeng")

Downloading:   0%|          | 0.00/820k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset lince downloaded and prepared to /home/jmperez/.cache/huggingface/datasets/lince/pos_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [25]:
from collections import Counter
pos_tags = lince_pos["train"]["pos"]

Counter(x for sentence in pos_tags for x in sentence).most_common()

[('VERB', 35297),
 ('PUNCT', 33922),
 ('PRON', 33243),
 ('NOUN', 19416),
 ('DET', 15361),
 ('ADV', 14570),
 ('ADP', 12515),
 ('INTJ', 10435),
 ('CONJ', 7684),
 ('ADJ', 7257),
 ('AUX', 7178),
 ('SCONJ', 6840),
 ('PART', 4576),
 ('PROPN', 4502),
 ('NUM', 2634),
 ('UNK', 1624),
 ('X', 14)]

In [26]:
%load_ext autoreload
%autoreload 2
from datasets import DatasetDict
from pysentimiento.lince.pos import load_datasets


train_dataset, dev_dataset, test_dataset = load_datasets("es")

lince_pos = DatasetDict(
    train=train_dataset,
    validation=dev_dataset,
    test=test_dataset,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Reusing dataset lince (/home/jmperez/.cache/huggingface/datasets/lince/pos_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/27893 [00:00<?, ?ex/s]

In [53]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer


device = "cuda" if torch.cuda.is_available() else "cpu"
max_length = 128

model_name = "pysentimiento/robertuito-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = max_length

In [54]:
from pysentimiento.training import load_model
from pysentimiento.lince.ner import label2id, id2label, tokenize_and_align_labels

base_model = "pysentimiento/robertuito-base-uncased"

model, tokenizer = load_model(
    base_model, label2id=label2id, id2label=id2label,
    max_length=max_length,
    auto_class=AutoModelForTokenClassification
)

tokenized_datasets = lince_ner.map(
    lambda x: tokenize_and_align_labels(x, tokenizer),
    batched=True,
    #remove_columns=lince_ner["train"].column_names,
)



Loading model pysentimiento/robertuito-base-uncased


Some weights of the model checkpoint at pysentimiento/robertuito-base-uncased were not used when initializing RobertaForTokenClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at pysentimiento/robertuito-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a dow

  0%|          | 0/34 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/24 [00:00<?, ?ba/s]

Sanity check

In [55]:
from tqdm.auto import tqdm

problematic = []

for split in tokenized_datasets:
    for idx, ex in tqdm(enumerate(tokenized_datasets[split]), total=len(tokenized_datasets[split])):
        if not (len(ex["words"]) == (ex["word_ids"][-2] + 1)):
            problematic.append((split, idx))

print(f"{len(problematic)} problematic examples")


  0%|          | 0/33611 [00:00<?, ?it/s]

  0%|          | 0/10085 [00:00<?, ?it/s]

  0%|          | 0/23527 [00:00<?, ?it/s]

0 problematic examples


In [61]:
assert model.config.num_labels == len(id2label)

In [62]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)



In [63]:
from datasets import load_metric

metric = load_metric("seqeval")

example = lince_ner["train"][1]

labels = example["ner"]
preds = labels.copy()
preds[0] = "B-LOC"

metric.compute(predictions=[preds], references=[labels])

/home/jmperez/.cache/pypoetry/virtualenvs/pysentimiento-bwlKzHxB-py3.7/lib/python3.7/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'LOC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 0.5,
 'overall_recall': 1.0,
 'overall_f1': 0.6666666666666666,
 'overall_accuracy': 0.9}

In [64]:
import numpy as np
from seqeval.metrics import f1_score


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    ret = {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "macro_f1": all_metrics["overall_f1"],
        "micro_f1": f1_score(true_labels, true_predictions, average="micro"),
        "accuracy": all_metrics["overall_accuracy"],
    }

    for k, v in all_metrics.items():
        if not k.startswith("overall"):
            ret[k + "_f1"] = v["f1"]
            ret[k + "_precision"] = v["precision"]
            ret[k + "_recall"] = v["recall"]

    return ret 


In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)